In [1]:
#Load libraries 

import pandas as pd
import sspa
import scipy
import numpy as np 
import random
import sys #to get the array job number when running an array job with the HPC

In [2]:
#Load dataset
df = pd.read_csv('Data/Su_COVID_metabolomics_processed.csv', index_col=0)
df.index= df.index.str.rstrip('-BL')

df2 = pd.read_csv('Data/Su_COVID_proteomics_processed.csv', index_col=0)

#Obtain common samples and subset accordingly
intersection = list(set(df.index.tolist()) & set(df2.index.tolist())) #set removes duplicates
intersection = [sample for sample in intersection if sample.startswith("INCOV")]
print(len(intersection))

df = df[df.index.isin(intersection)]

#Make a dictionary with the WHO status for each sample
sample_dict = {sample:df["WHO_status"][sample] for sample in df.index}

128


In [3]:
#Download the reactome pathways
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

#Download the root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())

In [12]:
sample_names = list(df.index)
random.shuffle(sample_names)
print(sample_names)

#Make a copy of the original dataframe but replace with the shuffled labels
df_shuffled = df.copy()
df_shuffled.index = sample_names

#Change the WHO status to match the shuffled label
for sample in df_shuffled.index: 
        df_shuffled.loc[sample,"WHO_status"] = sample_dict[sample]

display(df_shuffled)

df_mild = (df_shuffled[df_shuffled["WHO_status"] == '1-2']).iloc[:,:-2] #45 samples, remove the metadata
df_severe = (df_shuffled[(df_shuffled["WHO_status"] == '3-4') | (df_shuffled["WHO_status"] == '5-7')]).iloc[:,:-2] #83 samples

['INCOV031', 'INCOV008', 'INCOV131', 'INCOV075', 'INCOV045', 'INCOV052', 'INCOV013', 'INCOV096', 'INCOV134', 'INCOV139', 'INCOV060', 'INCOV102', 'INCOV084', 'INCOV091', 'INCOV072', 'INCOV054', 'INCOV066', 'INCOV068', 'INCOV067', 'INCOV040', 'INCOV071', 'INCOV079', 'INCOV061', 'INCOV142', 'INCOV092', 'INCOV093', 'INCOV027', 'INCOV034', 'INCOV038', 'INCOV050', 'INCOV048', 'INCOV126', 'INCOV025', 'INCOV109', 'INCOV023', 'INCOV112', 'INCOV116', 'INCOV115', 'INCOV021', 'INCOV011', 'INCOV106', 'INCOV123', 'INCOV111', 'INCOV002', 'INCOV082', 'INCOV020', 'INCOV001', 'INCOV127', 'INCOV053', 'INCOV018', 'INCOV019', 'INCOV077', 'INCOV098', 'INCOV055', 'INCOV004', 'INCOV043', 'INCOV103', 'INCOV024', 'INCOV094', 'INCOV133', 'INCOV128', 'INCOV017', 'INCOV026', 'INCOV051', 'INCOV086', 'INCOV078', 'INCOV122', 'INCOV135', 'INCOV137', 'INCOV125', 'INCOV097', 'INCOV076', 'INCOV037', 'INCOV108', 'INCOV059', 'INCOV081', 'INCOV120', 'INCOV009', 'INCOV124', 'INCOV080', 'INCOV101', 'INCOV056', 'INCOV049', 'IN

,1372,16610,72665,27823,30915,37373,16831,545959,17050,16359,...,133693,133694,28036,28238,76341,89312,17861,89188,WHO_status,Group
INCOV031,-0.351626,-0.455614,-0.717430,-0.546951,0.402428,-0.144621,0.254647,0.894678,-0.387132,-0.348626,...,-0.469802,-0.052650,-0.054083,0.093194,0.326376,2.008535,-0.001634,-0.846399,5-7,COVID19
INCOV008,-0.116658,-0.631844,0.576729,-0.943095,-0.809098,0.752301,-0.652306,0.099049,-0.395041,-0.479329,...,0.586373,0.511790,0.629809,-0.472752,0.153811,-0.852098,1.616299,0.323276,5-7,COVID19
INCOV131,-0.911383,-0.022064,4.301536,-0.647288,-0.823065,2.040576,0.894944,0.238973,-0.213630,-0.000006,...,2.262742,1.588177,-0.144172,-1.605203,-0.550098,2.620006,-1.029992,0.193321,1-2,COVID19
INCOV075,-0.590427,-1.205552,-1.381608,-1.583522,0.354906,0.131079,0.671972,-0.471912,-1.040489,-0.514651,...,0.893775,-0.140428,-0.229584,-0.963001,1.992081,2.546716,-0.952504,-0.655527,5-7,COVID19
INCOV045,-0.784259,0.797437,-0.726331,-0.820357,0.955426,1.012069,0.154660,-0.731348,-0.791854,-0.244991,...,-0.165858,-0.571452,-0.139356,0.253150,-0.264185,-0.204292,-0.414546,-1.087721,3-4,COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV041,-2.228940,-0.101002,-0.679609,0.000163,0.284214,0.734004,-0.041038,-0.210435,0.070459,-0.374044,...,-0.525115,-0.495566,-0.182214,0.714288,-0.265526,-0.151931,1.330576,-1.105986,5-7,COVID19
INCOV121,-0.169748,1.599117,0.173275,0.046243,-0.470655,1.219268,0.504222,1.204577,2.016911,0.431724,...,-0.768909,-0.829643,-0.247622,-1.579635,0.567450,-0.364475,1.220314,-0.755291,3-4,COVID19
INCOV015,0.317113,0.383352,-0.981562,0.857227,-1.462299,0.594609,0.561018,-0.709397,0.361510,-0.550916,...,-1.380566,-1.338709,2.431694,0.961980,0.630608,-0.262134,0.475143,-1.235598,3-4,COVID19
INCOV074,0.856401,1.847067,1.837225,2.220553,-0.167827,0.148805,0.901317,0.782439,2.592986,-0.374388,...,-1.104065,-1.005903,1.430243,-0.016018,-0.279534,1.291420,-1.179473,0.417561,1-2,COVID19


In [ ]:
#Function to calculate the squared Spearman correlation matrix 

def squared_spearman_corr(data):
    kpca_scores = sspa.sspa_kpca(data, reactome_pathways)   
    kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns))) #using Sara's code to drop root pathways

    spearman_results = scipy.stats.spearmanr(kpca_scores)
    squared_spearman_coef = np.square(spearman_results[0]) #correlation coefficients (spearman_results[1] gives the p-values)

    return squared_spearman_coef,list(kpca_scores.columns)




#Function to calculate the absolute difference between two matrices and then determine the mean for each edge

def delta_squared_list(data1,data2,edgelist):
    delta_squared = (np.array(data1) - np.array(data2))

    #Mask the upper half of the dataframe (so I don't view the comparisons between the two same genes, and also the duplicate comparisons are removed)
    mask = delta_squared.copy()
    mask = np.triu(np.ones(mask.shape)).astype(bool)
    mask = np.invert(mask) #invert true and false values so the diagonal is False as well
    non_dup_delta_squared = pd.DataFrame(delta_squared, columns = edgelist, index = edgelist)
    non_dup_delta_squared = pd.DataFrame(non_dup_delta_squared).where(mask) #Replace all false values with NaN using mask

    delta_squared_list = non_dup_delta_squared.stack().reset_index()
    finallist = list(delta_squared_list[0])

    return(finallist)

In [ ]:
spearman_mild,edgelist = squared_spearman_corr(df_mild)
spearman_severe,edgelist = squared_spearman_corr(df_severe)

output = delta_squared_list(spearman_mild,spearman_severe,edgelist)

In [ ]:
#Option 1: I can use the 'pickle' package to pickle the list
#Advantages: Stores as a list, which can be accessed upon reading in the file, and saves memory by about a half compared to saving as regular text file
#Disadvantages: Is not readable in text format (must be 'unpickled' in Python first), since it needs to be unpicked first the list cannot be indexed and all the files have to be read altogether, which requires a lot of memory and is computationally expensive (even the metabolomics data cannot handle it)

#with open('Results/Run'+index_num + '.txt', "wb") as file_output:  
#       pickle.dump(output,file_output)

#with open("file.txt", "rb") as file_input:   # Unpickling
#   list1 = pickle.load(file_input)

In [ ]:
index_num = sys.argv[1]  #this should return the array number within the array job

#Option 2: Write as a regular text file with a comma between values
with open ('Results/Run'+index_num+'.txt', 'w') as file:
     file.write(','.join(str(i) for i in output))